# 第2天：LangChain框架深入学习

## 今日学习目标
1. 掌握LangChain的核心架构
2. 深入理解Prompt Templates和Output Parsers
3. 实现多种Memory系统
4. 学习Document处理流程
5. 构建一个完整的对话系统

In [1]:
# 环境设置
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["LANGCHAIN_TRACING_V2"] = "false"

print("✅ 环境配置完成")

✅ 环境配置完成


## 1. LangChain核心架构理解

### LangChain的6大核心组件
1. **Models（模型）**：各种LLM的统一接口
2. **Prompts（提示词）**：管理和优化提示词
3. **Chains（链）**：组合多个组件的工作流
4. **Memory（记忆）**：维持对话状态和历史
5. **Agents（代理）**：让LLM选择和执行工具
6. **Callbacks（回调）**：监控和调试执行过程

In [5]:
# 核心导入
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser, PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory
from langchain.schema import BaseMessage
from typing import List

# 初始化基础模型
llm = ChatOpenAI(
    model="gpt-4.1-2025-04-14",
    temperature=0.7,
    max_tokens=1000
)

print("🤖 LLM模型初始化完成")

d:\Anaconda\envs\ai-guide\Lib\site-packages\IPython\core\interactiveshell.py:3699: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


🤖 LLM模型初始化完成


## 2. Prompt Templates深入实践

### 2.1 基础Prompt Template

In [7]:
# 基础提示词模板
basic_prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个专业的{role}，擅长{skill}。请用{style}的风格回答问题。"),
    ("human", "{question}")
])

# 创建基础链
basic_chain = basic_prompt | llm | StrOutputParser()

# 测试不同角色
print("=== 基础Prompt Template测试 ===")

test_cases = [
    {
        "role": "AI技术专家",
        "skill": "深度学习和机器学习",
        "style": "通俗易懂",
        "question": "什么是Transformer架构？"
    },
    {
        "role": "诗人",
        "skill": "现代诗创作",
        "style": "优美抒情",
        "question": "描述一下春天的景色"
    }
]

for i, case in enumerate(test_cases, 1):
    print(f"\n--- 测试 {i} ---")
    response = basic_chain.invoke(case)
    print(f"角色: {case['role']}")
    print(f"回答: {response[:200]}...")

=== 基础Prompt Template测试 ===

--- 测试 1 ---
角色: AI技术专家
回答: Transformer是一种深度学习模型架构，最早在2017年被Google提出，专门用来处理序列数据，比如自然语言（文本）。它革新了自然语言处理（NLP）领域，现在很多强大的AI模型（比如ChatGPT、BERT）都是基于Transformer的。

用通俗的话来说，Transformer就像是一个超级“翻译官”，能够理解和转换长短不一的文本、语音甚至代码。相比传统的循环神经网络（RNN）和卷积...

--- 测试 2 ---
角色: 诗人
回答: 春天是一首温柔的诗，  
在细雨和晨曦中慢慢展开。  
嫩绿的芽尖，悄悄探出泥土，  
像初醒的梦想，微微颤抖。  
桃花粉红，柳絮轻扬，  
风，带着泥土与花香，  
唤醒沉睡一冬的河流和鸟鸣。  
田野在阳光下渐渐铺开，  
小草偷偷地装点着大地的衣裳。  
一切都是崭新的，温柔的，  
仿佛每一寸空气里都藏着希望，  
在这个季节，连呼吸都是诗行。...


### 2.2 条件化Prompt Template

In [8]:
# 条件化提示词（根据用户类型调整）
def create_conditional_prompt(user_type: str) -> ChatPromptTemplate:
    """根据用户类型创建不同的提示词"""
    
    if user_type == "beginner":
        system_message = """
        你是一个耐心的AI导师，专门为初学者解答问题。
        请用简单易懂的语言解释，避免使用专业术语。
        如果必须使用术语，请提供简单的解释。
        用类比和例子帮助理解。
        """
    elif user_type == "expert":
        system_message = """
        你是一个技术专家，为有经验的开发者提供深入分析。
        可以使用专业术语和技术细节。
        提供代码示例和最佳实践。
        关注性能优化和架构设计。
        """
    else:  # intermediate
        system_message = """
        你是一个经验丰富的技术顾问。
        提供中等深度的技术解释。
        包含一些代码示例和实践建议。
        平衡理论和实践应用。
        """
    
    return ChatPromptTemplate.from_messages([
        ("system", system_message.strip()),
        ("human", "{question}")
    ])

# 测试条件化提示词
print("=== 条件化Prompt Template测试 ===")
question = "什么是RESTful API？"

for user_type in ["beginner", "intermediate", "expert"]:
    print(f"\n--- {user_type.upper()} 版本 ---")
    prompt = create_conditional_prompt(user_type)
    chain = prompt | llm | StrOutputParser()
    response = chain.invoke({"question": question})
    print(f"回答: {response[:300]}...")

=== 条件化Prompt Template测试 ===

--- BEGINNER 版本 ---
回答: 好的，让我们用最简单的方式来理解RESTful API。

**1. API是什么？**  
API 就像一个餐厅的菜单。你（客户端，比如手机App或网页）想点菜（获取或发送数据），但你不能直接进厨房（服务器）。你通过菜单（API）告诉服务员（程序）你要什么，服务员帮你把菜从厨房带出来。

**2. RESTful是什么意思？**  
RESTful 其实是一种设计API的方法和风格。REST 代表“表现层状态转移（Representational State Transfer）”，名字听起来有点复杂，但意思很简单：  
- 它用“网址”（URL）来指代资源（比如用户、商品、文章等）。
- 它...

--- INTERMEDIATE 版本 ---
回答: RESTful API（Representational State Transfer，表现层状态转移的应用程序接口）是一种基于HTTP协议、遵循REST架构风格的Web服务设计方式。它允许不同的系统通过标准的HTTP动词（如GET、POST、PUT、DELETE等）进行通信，常见于Web开发中前后端的数据交互。

### 1. 基本原理

REST强调资源(Resource)的概念，每个资源由一个URL唯一标识。客户端通过HTTP方法对资源进行操作：

| HTTP方法 | 作用             | 说明           |
|----------|---------------...

--- EXPERT 版本 ---
回答: RESTful API（Representational State Transfer Application Programming Interface）是一种基于REST架构风格设计的Web API。REST由Roy Fielding在2000年提出，是一种轻量级、无状态、以资源为中心的分布式系统架构。RESTful API广泛用于Web服务的数据交互，具有良好的可扩展性、易用性和与HTTP协议的高度契合。

## 核心概念

1. **资源（Resource）**
   - 一切皆资源。资源通常用URI（统一资源标识符）唯一标识，如`/users/123`。


## 3. Output Parsers：结构化输出

### 3.1 JSON Output Parser

In [17]:
# JSON输出解析器
json_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    你是一个技术分析师。请分析给定的技术概念，并以JSON格式返回结果。
    
    返回格式：
    {{
        "concept": "技术概念名称",
        "definition": "简短定义",
        "pros": ["优点1", "优点2"],
        "cons": ["缺点1", "缺点2"],
        "use_cases": ["使用场景1", "使用场景2"],
        "difficulty": "easy/medium/hard",
        "learning_time": "预估学习时间"
    }}
    """),
    ("human", "请分析：{concept}")
])

json_parser = JsonOutputParser()
json_chain = json_prompt | llm | json_parser

# 测试JSON解析
print("=== JSON Output Parser测试 ===")
concepts = ["Docker容器化", "GraphQL"]

for concept in concepts:
    print(f"\n--- 分析：{concept} ---")
    try:
        result = json_chain.invoke({"concept": concept})
        print(f"概念: {result['concept']}")
        print(f"定义: {result['definition']}")
        print(f"难度: {result['difficulty']}")
        print(f"学习时间: {result['learning_time']}")
        print(f"优点: {', '.join(result['pros'])}")
    except Exception as e:
        print(f"解析错误: {e}")

=== JSON Output Parser测试 ===

--- 分析：Docker容器化 ---
概念: Docker容器化
定义: Docker容器化是一种利用Docker平台将应用及其依赖打包在独立、可移植容器中的技术，实现环境一致性和高效部署。
难度: medium
学习时间: 1-2周（基础到入门应用）
优点: 应用部署和迁移速度快, 保证开发、测试、生产环境一致性, 资源利用率高，便于水平扩展, 易于微服务架构实现

--- 分析：GraphQL ---
概念: GraphQL
定义: GraphQL是一种用于API的查询语言以及相应的服务器端运行时，用于根据客户端需求返回数据。
难度: medium
学习时间: 1-2周（熟悉基本概念、语法和典型应用）
优点: 客户端可以精确获取所需数据，减少冗余传输, 单一端点统一管理多种资源, 强类型系统，易于前后端协作和文档生成, 高效实现聚合查询，减少多次请求


### 3.2 Pydantic Output Parser（类型安全）

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 定义Pydantic模型
class TechAnalysis(BaseModel):
    """技术分析结果模型"""
    concept: str = Field(description="技术概念名称")
    definition: str = Field(description="技术定义")
    pros: List[str] = Field(description="优点列表")
    cons: List[str] = Field(description="缺点列表")
    difficulty: str = Field(description="难度等级：easy/medium/hard")
    learning_time: str = Field(description="预估学习时间")
    related_technologies: List[str] = Field(description="相关技术")

# 创建Pydantic解析器
pydantic_parser = PydanticOutputParser(pydantic_object=TechAnalysis)

pydantic_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    你是一个技术分析专家。请分析给定的技术概念。
    
    {format_instructions}
    """),
    ("human", "请详细分析：{concept}")
])

pydantic_chain = pydantic_prompt | llm | pydantic_parser

# 测试Pydantic解析
print("=== Pydantic Output Parser测试 ===")
concept = "微服务架构"

try:
    result = pydantic_chain.invoke({
        "concept": concept,
        "format_instructions": pydantic_parser.get_format_instructions()
    })
    
    print(f"📋 分析结果类型: {type(result)}")
    print(f"🎯 概念: {result.concept}")
    print(f"📝 定义: {result.definition}")
    print(f"⭐ 优点: {result.pros}")
    print(f"⚠️ 缺点: {result.cons}")
    print(f"🎚️ 难度: {result.difficulty}")
    print(f"⏰ 学习时间: {result.learning_time}")
    print(f"🔗 相关技术: {result.related_technologies}")
    
except Exception as e:
    print(f"解析错误: {e}")

=== Pydantic Output Parser测试 ===
📋 分析结果类型: <class '__main__.TechAnalysis'>
🎯 概念: 微服务架构
📝 定义: 微服务架构是一种将单一应用程序划分为一组小的、独立的服务的方法，每个服务运行在其独立的进程中，并通过轻量级通信机制（通常是HTTP API）相互协作。每个微服务围绕着特定的业务能力构建，能够独立部署和扩展，通常由小型团队负责开发和维护。
⭐ 优点: ['易于扩展：每个服务可以独立伸缩，满足不同模块的性能需求。', '技术多样性：不同微服务可以使用最合适的技术栈和编程语言开发。', '高可用性：单个服务故障不会影响整个系统，提升系统容错能力。', '灵活部署：支持持续交付和持续部署，加快产品迭代速度。', '便于团队协作：小团队可独立负责特定微服务，减少沟通成本。']
⚠️ 缺点: ['系统复杂性增加：服务数量增多导致部署、运维和监控难度上升。', '分布式系统挑战：需要处理服务间通信、数据一致性、网络延迟等复杂问题。', '运维成本提升：需要引入服务注册、发现、配置管理、链路追踪等基础设施。', '测试困难：集成测试和端到端测试更加复杂，故障定位难度增加。', '数据管理复杂：跨服务的数据一致性和事务管理成为难点。']
🎚️ 难度: hard
⏰ 学习时间: 1-3个月（有分布式系统基础）
🔗 相关技术: ['容器化（Docker、Kubernetes）', '服务网格（Istio、Linkerd）', 'API网关（Kong、Nginx、Zuul）', '服务注册与发现（Consul、Eureka、Zookeeper）', '持续集成/持续部署（CI/CD）', '消息队列（Kafka、RabbitMQ、RocketMQ）', 'Spring Cloud', 'gRPC']


## 4. Memory系统：让AI记住对话

### 4.1 ConversationBufferMemory（完整记忆）

In [22]:
from langchain.chains import ConversationChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List

# 创建完整记忆系统
buffer_memory = ConversationBufferMemory()

conversation = ConversationChain(
    llm=llm,
    memory=buffer_memory,
    verbose=True
)

print("=== ConversationBufferMemory测试 ===")

# 模拟对话
conversation_topics = [
    "我的名字是张三，我是一名Python开发者",
    "我正在学习AI技术，特别是LangChain",
    "你还记得我的名字和职业吗？",
    "我之前提到我在学习什么技术？"
]

for i, topic in enumerate(conversation_topics, 1):
    print(f"\n--- 对话轮次 {i} ---")
    print(f"用户: {topic}")
    response = conversation.predict(input=topic)
    print(f"AI: {response}")
    
    # 显示记忆内容
    print(f"\n当前记忆:")
    print(buffer_memory.buffer)

=== ConversationBufferMemory测试 ===

--- 对话轮次 1 ---
用户: 我的名字是张三，我是一名Python开发者


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 我的名字是张三，我是一名Python开发者
AI:

> Finished chain.
AI: 你好，张三！很高兴认识你。作为一名Python开发者，你平时主要用Python做哪些方面的开发呢？比如Web开发、数据分析、自动化脚本、机器学习，还是其他的方向？如果你有任何技术上的问题，或者想要交流一些Python相关的经验和资源，随时可以告诉我哦！

当前记忆:
Human: 我的名字是张三，我是一名Python开发者
AI: 你好，张三！很高兴认识你。作为一名Python开发者，你平时主要用Python做哪些方面的开发呢？比如Web开发、数据分析、自动化脚本、机器学习，还是其他的方向？如果你有任何技术上的问题，或者想要交流一些Python相关的经验和资源，随时可以告诉我哦！

--- 对话轮次 2 ---
用户: 我正在学习AI技术，特别是LangChain


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its co

### 4.2 ConversationBufferWindowMemory（滑动窗口记忆）

In [ ]:
# 创建滑动窗口记忆（只记住最近3轮对话）
window_memory = ConversationBufferWindowMemory(k=3)

window_conversation = ConversationChain(
    llm=llm,
    memory=window_memory,
    verbose=False
)

print("\n=== ConversationBufferWindowMemory测试 ===")
print("只记住最近3轮对话")

# 进行多轮对话测试
test_messages = [
    "第1轮：我喜欢蓝色",
    "第2轮：我的爱好是编程", 
    "第3轮：我住在北京",
    "第4轮：我今年25岁",
    "第5轮：我学过Java",
    "第6轮：你还记得我喜欢什么颜色吗？"  # 应该不记得，因为超出窗口
]

for msg in test_messages:
    response = window_conversation.predict(input=msg)
    print(f"\n{msg}")
    print(f"AI: {response[:100]}...")
    print(f"记忆轮次: {len(window_memory.chat_memory.messages)//2}")


=== ConversationBufferWindowMemory测试 ===
只记住最近3轮对话

第1轮：我喜欢蓝色
AI: 蓝色真是一个很美的颜色！它常常让人联想到天空和大海，给人一种宁静、清新的感觉。你喜欢蓝色的哪一种呢？比如说天蓝、宝石蓝、藏青色，还是其他什么蓝色？你有没有喜欢的蓝色物品，比如衣服、文具，或者房间的装饰...
记忆轮次: 1

第2轮：我的爱好是编程
AI: 编程真棒！它不仅能锻炼逻辑思维，还能让你创造出各种有趣的项目。你是怎么开始接触编程的呢？你平时用什么编程语言比较多？比如 Python、Java、C++ 还是 JavaScript？你有没有做过一些让...
记忆轮次: 2

第3轮：我住在北京
AI: 北京是一座充满历史与现代气息的城市！这里既有壮丽的古迹，比如故宫、天坛、长城，也有繁华的都市生活，比如三里屯、国贸、798艺术区。北京的小吃也很有特色，像豆汁儿、炸酱面、卤煮火烧、冰糖葫芦，都让人难忘...
记忆轮次: 3

第4轮：我今年25岁
AI: 25岁正是充满活力和无限可能的年纪！这个阶段，很多人会在学习、工作、生活中不断探索和成长。你现在是在上学、刚开始工作，还是已经有一些职业经验了呢？有没有什么目标或者梦想是你特别想实现的？

其实，25...
记忆轮次: 4

第5轮：我学过Java
AI: 太棒了！Java 是一门非常实用而且流行的编程语言，无论是在企业级开发、移动应用（比如 Android）、还是后端开发中都非常常见。你学 Java 有多久了？主要是自学、在学校课程中学的，还是工作中用...
记忆轮次: 5

第6轮：你还记得我喜欢什么颜色吗？
AI: 当然记得！你喜欢蓝色。蓝色是一个很受欢迎的颜色，既有清新的天蓝，也有深邃的宝石蓝和藏青色。之前我们还聊到过关于蓝色有什么不同的色调和你喜欢的蓝色物品。如果你还想聊更多和蓝色相关的话题，比如蓝色的搭配、...
记忆轮次: 6


### 4.3 ConversationSummaryMemory（摘要记忆）

In [6]:
from langchain.chains import ConversationChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field
from typing import List
from langchain.memory import ConversationSummaryMemory

# 创建摘要记忆系统
summary_memory = ConversationSummaryMemory(
    llm=llm,
    max_token_limit=200  # 当记忆超过1000 tokens时开始摘要
)

summary_conversation = ConversationChain(
    llm=llm,
    memory=summary_memory,
    verbose=False
)

print("\n=== ConversationSummaryMemory测试 ===")

# 模拟长对话
long_conversation = [
    "我是一名AI研究员，专注于自然语言处理",
    "我最近在研究大语言模型的微调技术",
    "我发现LoRA是一个很有效的参数高效微调方法",
    "我还尝试过QLoRA，它在内存使用上更优化",
    "除了微调，我也在研究RAG技术",
    "RAG可以让模型获取最新的外部知识",
    "你能总结一下我们刚才讨论的内容吗？"
]

for i, msg in enumerate(long_conversation, 1):
    response = summary_conversation.predict(input=msg)
    print(f"\n轮次 {i}: {msg}")
    print(f"AI: {response[:150]}...")
    
    # 显示摘要
    if hasattr(summary_memory, 'buffer'):
        print(f"摘要: {summary_memory.buffer[:100]}...")

C:\Users\18889\AppData\Local\Temp\ipykernel_31376\2700894171.py:10: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(
C:\Users\18889\AppData\Local\Temp\ipykernel_31376\2700894171.py:15: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :class:`~langchain_core.runnables.history.RunnableWithMessageHistory` instead.
  summary_conversation = ConversationChain(



=== ConversationSummaryMemory测试 ===


Failed to multipart ingest runs: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')



轮次 1: 我是一名AI研究员，专注于自然语言处理
AI: 太棒了！很高兴认识你。自然语言处理（NLP）是人工智能领域最活跃、最有前景的研究方向之一。你最近在关注哪些NLP子领域呢？比如大语言模型、对话系统、信息抽取、文本生成，还是多模态学习？如果你愿意分享的话，也很想听听你目前参与的项目或者遇到的有趣挑战。...
摘要: A human介绍自己是一名专注于自然语言处理的AI研究员。AI表示很高兴认识对方，并指出自然语言处理是人工智能领域中最活跃、最有前景的方向之一。AI还询问对方最近关注的NLP子领域，以及目前参与的项...


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


...


轮次 4: 我还尝试过QLoRA，它在内存使用上更优化
AI: 是的，QLoRA（Quantized LoRA）确实在内存和计算资源使用方面有了进一步的优化。QLoRA的核心创新在于结合了参数高效微调（如LoRA）和权重量化技术。具体来说，QLoRA通过将基础大语言模型的权重进行低比特（如4-bit）量化，大幅压缩了模型在微调和推理时的内存占用；同时，在量化模型...
摘要: A human介绍自己是一名专注于自然语言处理的AI研究员，AI表示很高兴认识对方，并指出NLP是人工智能领域中最活跃、最有前景的方向之一，询问对方当前关注的NLP子领域及项目挑战。Human表示自己...


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


KeyboardInterrupt: 

Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')


Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTPError('403 Client Error: Forbidden for url: https://api.smith.langchain.com/runs/multipart', '{"error":"Forbidden"}\n')
Failed to send compressed multipart ingest: langsmith.utils.LangSmithError: Failed to POST https://api.smith.langchain.com/runs/multipart in LangSmith API. HTTP

## 5. Document Loaders和Text Splitters

### 5.1 处理不同格式的文档

In [7]:
from langchain.text_splitter import (
    RecursiveCharacterTextSplitter, 
    CharacterTextSplitter,
    TokenTextSplitter
)
from langchain.document_loaders import TextLoader
from langchain.schema import Document

# 创建示例文档
sample_text = """
人工智能（Artificial Intelligence, AI）是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。

机器学习（Machine Learning, ML）是人工智能的一个子集，它使计算机能够在没有明确编程的情况下学习和改进。机器学习算法使用统计技术来找到数据中的模式。

深度学习（Deep Learning, DL）是机器学习的一个子集，它使用人工神经网络来模拟人脑的学习过程。深度学习在图像识别、自然语言处理和语音识别等领域取得了突破性进展。

自然语言处理（Natural Language Processing, NLP）是人工智能的一个重要分支，专注于让计算机理解、解释和生成人类语言。NLP结合了计算语言学、机器学习和深度学习技术。

大语言模型（Large Language Models, LLMs）如GPT、BERT等，代表了NLP领域的最新进展。这些模型在大量文本数据上进行预训练，能够理解和生成高质量的自然语言文本。
"""

# 创建文档对象
doc = Document(page_content=sample_text, metadata={"source": "AI技术概述"})

print("=== 文档处理测试 ===")
print(f"原始文档长度: {len(sample_text)} 字符")
print(f"原始文档:\n{sample_text[:200]}...")

=== 文档处理测试 ===
原始文档长度: 441 字符
原始文档:

人工智能（Artificial Intelligence, AI）是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。

机器学习（Machine Learning, ML）是人工智能的一个子集，它使计算机能够在没有明确编程的情况下学习和改进。机器学习算法使用统计技术来找到数据中的模式。

深度学习（Deep Learning, DL）是机器学习的一个子集，它使用人工神经网络...


### 5.2 不同的文本分割策略

In [8]:
# 1. 递归字符分割器（推荐）
recursive_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,  # 每块最大字符数
    chunk_overlap=50,  # 重叠字符数
    separators=["\n\n", "\n", "。", "！", "？", ";", ",", " ", ""]  # 分割优先级
)

recursive_chunks = recursive_splitter.split_documents([doc])

print("\n=== 递归字符分割器 ===")
print(f"分割后块数: {len(recursive_chunks)}")
for i, chunk in enumerate(recursive_chunks):
    print(f"\n块 {i+1} (长度: {len(chunk.page_content)})")
    print(f"{chunk.page_content[:150]}...")

# 2. 基于Token的分割器
token_splitter = TokenTextSplitter(
    chunk_size=100,  # 每块最大token数
    chunk_overlap=20
)

token_chunks = token_splitter.split_documents([doc])

print(f"\n=== Token分割器 ===")
print(f"分割后块数: {len(token_chunks)}")
for i, chunk in enumerate(token_chunks[:2]):  # 只显示前两块
    print(f"\n块 {i+1}:")
    print(f"{chunk.page_content[:150]}...")


=== 递归字符分割器 ===
分割后块数: 3

块 1 (长度: 154)
人工智能（Artificial Intelligence, AI）是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。

机器学习（Machine Learning, ML）是人工智能的一个子集，它使计算机能够在没有明确编程的情况下学习和改进。机器学习算法使用统计技术来找到数据中...

块 2 (长度: 186)
深度学习（Deep Learning, DL）是机器学习的一个子集，它使用人工神经网络来模拟人脑的学习过程。深度学习在图像识别、自然语言处理和语音识别等领域取得了突破性进展。

自然语言处理（Natural Language Processing, NLP）是人工智能的一个重要分支，专注于让计算机理...

块 3 (长度: 95)
大语言模型（Large Language Models, LLMs）如GPT、BERT等，代表了NLP领域的最新进展。这些模型在大量文本数据上进行预训练，能够理解和生成高质量的自然语言文本。...

=== Token分割器 ===
分割后块数: 9

块 1:

人工智能（Artificial Intelligence, AI）是计算机科学的一个分支，致力于创建能够执行通常需要人类智能的任务的系统。

�...

块 2:
��能的任务的系统。

机器学习（Machine Learning, ML）是人工智能的一个子集，它使计算机能够在没有明确编程的情况下学�...


## 6. 综合实践：构建智能对话助手

将今天学到的所有技术整合到一个完整的对话系统中

In [9]:
class SmartConversationAssistant:
    """智能对话助手 - 整合今日所学技术"""
    
    def __init__(self):
        self.llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
        self.memory = ConversationBufferWindowMemory(k=5)  # 记住最近5轮对话
        
        # 智能提示词模板
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """
            你是一个智能AI助手，具有以下能力：
            1. 记住对话历史，提供连贯的回答
            2. 根据用户问题类型调整回答风格
            3. 提供结构化的信息分析
            
            对话历史：{history}
            
            请根据用户的问题类型选择合适的回答方式：
            - 技术问题：提供详细解释和代码示例
            - 概念解释：用简单易懂的语言和类比
            - 个人对话：记住用户信息，提供个性化回答
            """),
            ("human", "{input}")
        ])
        
        # 创建对话链
        self.conversation = ConversationChain(
            llm=self.llm,
            memory=self.memory,
            prompt=self.prompt,
            verbose=False
        )
    
    def chat(self, message: str) -> str:
        """进行对话"""
        try:
            response = self.conversation.predict(input=message)
            return response
        except Exception as e:
            return f"抱歉，处理出错：{str(e)}"
    
    def get_memory_summary(self) -> str:
        """获取记忆摘要"""
        return self.memory.buffer if hasattr(self.memory, 'buffer') else "暂无对话记录"

# 创建智能助手
assistant = SmartConversationAssistant()

print("=== 智能对话助手测试 ===")

# 测试对话场景
test_conversations = [
    "你好，我是小明，我想学习Python编程",
    "我对机器学习很感兴趣，但不知道从哪里开始",
    "你还记得我的名字吗？我想学习什么？",
    "能给我推荐一些Python机器学习的学习路径吗？",
    "谢谢你的建议，我会按照这个路径学习的"
]

for i, msg in enumerate(test_conversations, 1):
    print(f"\n--- 对话 {i} ---")
    print(f"用户: {msg}")
    response = assistant.chat(msg)
    print(f"助手: {response[:200]}...")

print(f"\n=== 对话记忆摘要 ===")
print(assistant.get_memory_summary()[:300] + "...")

C:\Users\18889\AppData\Local\Temp\ipykernel_31376\756752796.py:6: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  self.memory = ConversationBufferWindowMemory(k=5)  # 记住最近5轮对话


=== 智能对话助手测试 ===

--- 对话 1 ---
用户: 你好，我是小明，我想学习Python编程
助手: 你好小明，很高兴能帮助你学习Python编程。你已经有编程经验了吗？还是从零开始学习呢？如果你有任何关于Python编程的问题，都可以随时问我哦！...

--- 对话 2 ---
用户: 我对机器学习很感兴趣，但不知道从哪里开始
助手: 你好！很高兴听到你对机器学习感兴趣。作为一个初学者，你可以从以下几个步骤开始学习机器学习：

1. 掌握基础数学知识：机器学习涉及到很多数学概念，特别是线性代数、微积分和概率论等。建议你先打好数学基础。

2. 学习编程语言：Python是机器学习领域最常用的编程语言之一，你可以从学习Python开始。掌握Python基础语法和相关库（如NumPy、Pandas、Matplotlib等）对于机器学...

--- 对话 3 ---
用户: 你还记得我的名字吗？我想学习什么？
助手: 当然记得！你是小明，你对机器学习很感兴趣，但不知道从哪里开始学习。如果你想了解如何开始学习机器学习，我可以给你一些建议哦！...

--- 对话 4 ---
用户: 能给我推荐一些Python机器学习的学习路径吗？
助手: 你好！很高兴听到你对Python机器学习感兴趣。作为一个初学者，你可以从以下几个步骤开始学习Python机器学习：

1. 掌握基础数学知识：机器学习涉及到很多数学概念，特别是线性代数、微积分和概率论等。建议你先打好数学基础。

2. 学习编程语言：Python是机器学习领域最常用的编程语言之一，你可以从学习Python开始。掌握Python基础语法和相关库（如NumPy、Pandas、Matpl...

--- 对话 5 ---
用户: 谢谢你的建议，我会按照这个路径学习的
助手: 不客气，很高兴能帮助到你！如果在学习过程中有任何问题或需要进一步的指导，都可以随时向我咨询。祝你学习顺利，掌握机器学习的知识和技能！如果有其他问题，也欢迎随时和我交流。...

=== 对话记忆摘要 ===
Human: 你好，我是小明，我想学习Python编程
AI: 你好小明，很高兴能帮助你学习Python编程。你已经有编程经验了吗？还是从零开始学习呢？如果你有任何关于Python编程的问题，都可以随时问我哦！
Human: 我对

## 7. 今日学习总结

### ✅ 掌握的核心技能

1. **Prompt Templates**
   - 基础模板设计
   - 条件化提示词
   - 角色和风格控制

2. **Output Parsers**
   - JSON结构化输出
   - Pydantic类型安全解析
   - 自定义输出格式

3. **Memory Systems**
   - ConversationBufferMemory（完整记忆）
   - ConversationBufferWindowMemory（滑动窗口）
   - ConversationSummaryMemory（智能摘要）

4. **Document Processing**
   - 文档加载和处理
   - 多种文本分割策略
   - 块大小和重叠优化

### 🎯 关键收获

- **LangChain的模块化设计**：每个组件都可以独立使用和组合
- **链式调用的威力**：`prompt | llm | parser` 的简洁语法
- **Memory让对话更智能**：记住上下文，提供连贯体验
- **结构化输出的重要性**：便于后续处理和集成

### 📝 明日预告：第3天 - RAG核心技术

- 向量数据库深入实践
- 文档嵌入和相似度搜索
- 构建完整的RAG系统
- 检索质量优化技巧